In [10]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
from scipy import interpolate
import math
from matplotlib import ticker, cm, colors 
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
from numba import jit
from numba import prange
from scipy.special import sph_harm
from scipy.optimize import minimize
import time
import glob
import warnings 
warnings.filterwarnings('ignore')
import camb

OSError: dlopen(/Users/kokoorikunihiko/Library/Python/3.7/lib/python/site-packages/camb/camblib.so, 6): Library not loaded: /usr/local/opt/gcc/lib/gcc/9/libgfortran.5.dylib
  Referenced from: /Users/kokoorikunihiko/Library/Python/3.7/lib/python/site-packages/camb/camblib.so
  Reason: image not found

In [14]:
pars = camb.CAMBparams()
pars.set_cosmology(H0=67.5, ombh2=0.022, omch2=0.122) 
pars.InitPower.set_params(ns=0.965) 
pars.set_matter_power(redshifts=[0],kmax=100) 
results= camb.get_results(pars)
trans = results.get_matter_transfer_data()
kh = trans.transfer_data[0,:,0]
P_k_ini = interpolate.interp1d(kh, results.Params.scalar_power(kh), kind='linear')

NameError: name 'camb' is not defined

In [15]:
plt.loglog(kh,P_k_ini(kh))

NameError: name 'kh' is not defined

In [16]:
transfer_model = model.Transfer_cdm
pars = camb.CAMBparams()
pars.set_cosmology(H0=67.5, ombh2=0.022, omch2=0.122) pars.InitPower.set_params(ns=0.965)
pars.set_matter_power(redshifts=[0],kmax=100)
results= camb.get_results(pars)
trans = results.get_matter_transfer_data()
transfer = trans.transfer_data[transfer_model-1,:,0]
kh = trans.transfer_data[0,:,0]
transfer_function_cdm = interpolate.interp1d(kh, transfer, kind='linear')
P_k_ini = interpolate.interp1d(kh, results.Params.scalar_power(kh), kind='linear') def P_k(k):
return (P_k_ini(k)*(transfer_function_pot(k)**2)*(k**4))*(results.hubble_parameter(0)*0.01)**3/(k**3/(2*np.p i**2))

SyntaxError: invalid syntax (<ipython-input-16-061ecfa9c06d>, line 3)

In [18]:
plt.loglog(kh,2*np.pi**2*transfer_function_cdm(kh)**2*(P_k_ini(kh))*kh) 
plt.scatter(kh,np.random.normal(0,2*np.pi**2*transfer_function_cdm(kh)**2*(P_k_ini(kh))*kh))

NameError: name 'kh' is not defined

In [19]:
Omega_m = 0.3
H0 = pars.H0*np.sqrt(Omega_m)*(1+1000)*1.5 H0 = pars.H0
def gravitational_power(k):
return (9/4)*(Omega_m**2)*((H0*3*1e-5)**2)*P_k_ini(k)/k**4

SyntaxError: invalid syntax (<ipython-input-19-ebcf892dc77e>, line 2)

In [20]:
plt.loglog(kh,gravitational_power(kh))

NameError: name 'kh' is not defined

In [ ]:
transfer_pig = np.loadtxt("transfer/test_transfer_z01_pig.csv", delimiter=" ", skiprows=0, usecols=14) 
kh = np.loadtxt("transfer/test_transfer_z01_pig.csv", delimiter=" ", skiprows=0, usecols=1)*0.675 
transfer_function = interpolate.interp1d(kh, transfer_pig, kind='linear') 
#plt.semilogx(kh,transfer_function(kh))
def make_QU(bins,grid,L):
k_posi = np.arange(0,bins,1)*(2*np.pi/L) 
k_nega = np.arange(-bins,0,1)*(2*np.pi/L) 
k = np.hstack((k_posi, k_nega))
k_vec = np.zeros((3,grid))
#norm = np.zeros(grid,grid,bins)
for i in range(3):
k_vec[i,:] = k
norm = np.sqrt((k_vec[0]**2)[:,None,None]+(k_vec[1]**2)[None,:,None]+(k_vec[2]**2)[None,None,:]) 
#norm = norm[:,:,:bins]
inner_x = (k_vec[0])[:,None,None]/norm
inner_y = (k_vec[1])[None,:,None]/norm
inner_z = (k_vec[2])[None,None,:]/norm 
inner_x[0,0,0]=0
inner_y[0,0,0]=0
inner_z[0,0,0]=0
norm[0,0,0]=1e-3 np.random.seed(3)
theta = 2*np.random.rand(grid,grid,bins)*np.pi#128,256,256
#波数空間上での\phi,\theta
Y_theta = np.arccos(inner_z)#\theta
Y_phi = np.arccos(inner_x/np.sin(Y_theta))#\phi for i in range(grid):
for k in range(grid):
if np.isnan(Y_phi[i,0,k]) == True:
Y_phi[i,0,k] = 0 np.random.seed(4)
initial = np.random.normal(0,np.sqrt(P_k_ini(norm[:,:,:bins])))*np.exp(1j*theta) 
k_box = np.zeros((5,grid,grid,grid),dtype=complex)
delta_k = np.sqrt(2*(np.pi**2)*norm[:,:,:bins])*transfer_function(norm[:,:,:bins])*initial
k_box_harm = np.zeros((5,grid,grid,grid),dtype=complex) 
for m in range(5):
    Y_2m = np.zeros((grid,grid,grid),dtype=complex)
    Y_2m[:,:,:bins] = sph_harm(m-2, 2, Y_phi[:,:,:bins], Y_theta[:,:,:bins]).conjugate() 
    k_box[m,1:,1:,1:bins] = delta_k[1:,1:,1:bins] 
    k_box[m,grid:0:-1,grid:0:-1,grid:bins:-1] = k_box[m,1:,1:,1:bins].conjugate() 
    Y_2m[grid:0:-1,grid:0:-1,grid:bins:-1] = Y_2m[1:,1:,1:bins]
    #平面状の数値生成
    #kz-kx平面上
    k_box[m,:,0,1:bins] = delta_k[:,0,1:bins]
    k_box[m,grid:0:-1,0,grid:bins:-1] = k_box[m,1:,0,1:bins].conjugate() 
    Y_2m[grid:0:-1,0,grid:bins:-1] = Y_2m[1:,0,1:bins]
    #kx-ky平面上
    k_box[m,:,1:bins,0]= delta_k[:,1:bins,0]
    k_box[m,grid:0:-1,grid:bins:-1,0] = k_box[m,1:,1:bins,0].conjugate() 
    Y_2m[grid:0:-1,grid:bins:-1,0] = Y_2m[1:,1:bins,0]
    #ky-kz平面上
    k_box[m,0,:,1:bins] = delta_k[0,:,1:bins]
    k_box[m,0,grid:0:-1,grid:bins:-1] = k_box[m,0,1:,1:bins].conjugate() 
    Y_2m[0,grid:0:-1,grid:bins:-1] = Y_2m[0,1:,1:bins]
    #z軸上
    k_box[m,0,0,1:bins] = delta_k[0,0,1:bins]
    k_box[m,0,0,grid:bins:-1] = k_box[m,0,0,1:bins].conjugate()
    Y_2m[0,0,grid:bins:-1] = Y_2m[0,0,1:bins]
    #y軸上
    k_box[m,0,1:bins,0]= delta_k[0,1:bins,0]
    k_box[m,0,grid:bins:-1,0] = k_box[m,0,1:bins,0].conjugate()
    Y_2m[0,grid:bins:-1,0] = Y_2m[0,1:bins,0]
    #x軸上
    k_box[m,1:bins,0,0] = delta_k[1:bins,0,0]
    k_box[m,grid:bins:-1,0,0] = k_box[m,1:bins,0,0].conjugate()
    Y_2m[grid:bins:-1,0,0] = Y_2m[1:bins,0,0]


    k_box_harm[m,:,:,:] = Y_2m*k_box[m,:,:,:]
k_box_harm[:,bins,:,:] = 0 
k_box_harm[:,:,bins,:] = 0 
k_box_harm[:,:,:,bins] = 0 
k_box_harm[:,0,0,0] = 0